In [ ]:
import pandas as pd
import scipy
from scipy import stats
import numpy as np
from pylab import *
%matplotlib inline
import seaborn
import seaborn as sns
sns.set(color_codes=True)
import numpy

In [ ]:
# set to true for experiments on directed graphs, to false for experiments on undirected graphs
isDirected = True

undirected = ["Mus_musculus.txt.graph","HC-BIOGRID.txt.graph","Caenorhabditis_elegans.txt.graph","ca-GrQc.txt.graph","advogato.txt.graph","hprd_pp.txt.graph",
"ca-HepTh.txt.graph","Drosophila_melanogaster.txt.graph","oregon1_010526.txt.graph","oregon2_010526.txt.graph","Homo_sapiens.txt.graph","GoogleNw.txt.graph",
"CA-CondMat.txt"]

directed = ["out.subelj_jung-j_jung-j.graph", "wiki-Vote.txt.graph", "out.elec.graph", "freeassoc.txt.graph",  
"out.dblp-cite.graph", "out.subelj_cora_cora.graph", "out.ego-twitter.graph",  "out.ego-gplus.graph",
"out.munmun_digg_reply.graph"]

labelsUndirected = ["Mus-musculus","HC-BIOGRID","Caenor-eleg","ca-GrQc","advogato","hprd-pp",
    "ca-HepTh","dr-melanog","oregon1","oregon2","Homo-sapiens","GoogleNw","CA-CondMat"]

labelsDirected = ["subelj-jung", "wiki-Vote", "elec", "freeassoc", "dblp-cite", "subelj-cora", "ego-twitter", "ego-gplus",
         "munmun-digg"]

pathUndirected = "../scripts/resultsCompUndirected/results_"
pathDirected = "../scripts/resultsCompDirected/results_"

In [ ]:
def avgTimes(graphs, path, labels):
    # compute average times and std deviations
    # table 2 and 3 (depending on graphs and path)
    avgDyn = {}
    avgStat = {}
    avgDynOne = {}
    devDyn = {}
    devStat = {}
    devDynOne = {}


    for g in graphs:
        file = path+g+".csv"
        table = pd.read_csv(file)
        timesDyn = list(table["Dyn"])
        timesDynOne = list(table["Dyn One Node"])
        timesStat = list(table["Static bc"])
        avgDyn[g] = '{0:.4f}'.format(scipy.average(timesDyn))
        avgDynOne[g] = '{0:.4f}'.format(scipy.average(timesDynOne))
        avgStat[g] = '{0:.2f}'.format(scipy.average(timesStat))
        devDyn[g] = '{0:.4f}'.format(scipy.std(timesDyn))
        devDynOne[g] = '{0:.4f}'.format(scipy.std(timesDynOne))
        devStat[g] = '{0:.4f}'.format(scipy.std(timesStat))

    for i, g in enumerate(graphs):
        print("\\texttt{" + labels[i] + "} \t & \t" + str(avgStat[g])+"\t & \t"+str(avgDyn[g])+"\t & \t \\textbf{"+str(avgDynOne[g])+
              "}\t & \t"+str(devDyn[g])+"\t & \t"+str(devDynOne[g])+" \\\\")

In [ ]:
# print table 2 and table 3
#print("Results for directed graphs")
#avgTimes(directed, pathDirected, labelsDirected)
print("\n\nResults for undirected graphs\n")
avgTimes(undirected, pathUndirected, labelsUndirected)

In [ ]:
def printSpeedups(graphs, path, labels):
# now we print the geometric mean of the speedups on static recomputation and on the dynamic algo for all nodes, 
# the maximum and the minimum speedup (table 4 and 5)
    speedOnStat = {}
    speedOnDyn = {}
    maxSpeedOnStat = {}
    maxSpeedOnDyn = {}
    minSpeedOnStat = {}
    minSpeedOnDyn = {}

    for g in graphs:
        file = path+g+".csv"
        table = pd.read_csv(file)
        nSamples = len(table)
        timesDyn = []
        timesDynOne = []
        timesStat = []
        for i in range(nSamples):
            timesDyn.append(table["Dyn"][i])
            timesDynOne.append(table["Dyn One Node"][i])
            timesStat.append(table["Static bc"][i])
        speedOnStat[g] = '{0:.1f}'.format(stats.gmean([timesStat[i]/timesDynOne[i] for i in range(len(timesDynOne))]))
        speedOnDyn[g] = '{0:.1f}'.format(stats.gmean([timesDyn[i]/timesDynOne[i] for i in range(len(timesDynOne))]))
        maxSpeedOnStat[g] = '{0:.1f}'.format(max([timesStat[i]/timesDynOne[i] for i in range(len(timesDynOne))]))
        maxSpeedOnDyn[g] = '{0:.1f}'.format(max([timesDyn[i]/timesDynOne[i] for i in range(len(timesDynOne))]))
        minSpeedOnStat[g] = '{0:.1f}'.format(min([timesStat[i]/timesDynOne[i] for i in range(len(timesDynOne))]))
        minSpeedOnDyn[g] = '{0:.1f}'.format(min([timesDyn[i]/timesDynOne[i] for i in range(len(timesDynOne))]))

    for i, g in enumerate(graphs):
        print("\\texttt{" + labels[i] + "} \t & \t" + speedOnStat[g]+"\t & \t"+maxSpeedOnStat[g]+"\t & \t"+minSpeedOnStat[g]+"\t & \t"+speedOnDyn[g]+
              "\t & \t"+maxSpeedOnDyn[g]+"\t & \t"+minSpeedOnDyn[g]+" \\\\")

In [ ]:
# print table 4 and table 5
print("Results for directed graphs")
printSpeedups(directed, pathDirected, labelsDirected)
print("\n\nResults for undirected graphs\n")
printSpeedups(undirected, pathUndirected, labelsUndirected)

In [ ]:
# figure 13, 14 (change g to see different graphs)
# set zoom to 0 for no zoom, or to a specific value to zoom in the interval [0, zoom] in the y axis
def showAffected(g, path, zoom):
    file = path+g+".csv"
    table = pd.read_csv(file)


    # we sort by affected nodes
    dyn = [(table["Affected"][i], table["Dyn"][i]) for i in range(len(table["Affected"]))]
    dyn.sort()
    dynOne = [(table["Affected"][i], table["Dyn One Node"][i]) for i in range(len(table["Affected"]))]
    dynOne.sort()

    x = [dyn[i][0] for i in range(len(dyn))]
    y1 = [dyn[i][1] for i in range(len(dyn))]
    y2 = [dynOne[i][1] for i in range(len(dynOne))]
    y3 = list(table["Static bc"])

    # plot
    f, ax = plt.subplots(1, 1)

    np.random.seed(8)
    mean, cov = [4, 6], [(1.5, .7), (.7, 1)]
    x = array(x)
    y1 = array(y1)
    y2 = array(y2)
    seaborn.regplot(x=x, y=y1, color="r", label="AI")
    seaborn.regplot(x=x, y=y2, color="b", label="SI")
    ax.legend(("AI", "SI"), fontsize = 10)
    xlabel('Affected pairs')
    ylabel('Time [s]')
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(10)

    xlim(0, max(x))
    if zoom > 0:
        ylim(0, zoom)
    show()

In [ ]:
# figure 13 (top left)
showAffected(directed[7], pathDirected, 0)

In [ ]:
# figure 13 (bottom left)
showAffected(directed[7], pathDirected, 0.1)

In [ ]:
# figure 13 (top right)
showAffected(directed[8], pathDirected, 0)

In [ ]:
# figure 13 (bottom right)
showAffected(directed[8], pathDirected, 5.00)

In [ ]:
# figure 14 (top left)
showAffected(undirected[7], pathUndirected, 0)

In [ ]:
# figure 14 (bottom left)
showAffected(undirected[7], pathUndirected, 2)

In [ ]:
# figure 14 (top right)
showAffected(undirected[10], pathUndirected, 1)

In [ ]:
# figure 14 (bottom right)
showAffected(undirected[7], pathUndirected, 2.0)

In [ ]:
# Comparison with other heuristics (Figures 7 - 12)
# we first read the necessary data 
    
def readDataHeuristics(file):
    f = open(file, "r")
    init_scores = {}
    init_ranks = {}
    scores_greedy = {}
    ranks_greedy = {}
    scores_bc = {}
    ranks_bc = {}
    scores_deg = {}
    ranks_deg = {}
    scores_rand = {}
    ranks_rand = {}
    nodes = {}
    edges = {}
    current = "x"
    sample = 0
    for line in f:
        if line.startswith("Graph,"):
            continue
        if line.startswith("Graph:"):
            graph = line.split(",")[0].split(":")[1].strip()
            if graph != current:
                init_scores[graph] = [[]]
                init_ranks[graph] = [[]]
                scores_greedy[graph] = [[]]
                ranks_greedy[graph] = [[]]
                scores_bc[graph] = [[]]
                ranks_bc[graph] = [[]]
                scores_deg[graph] = [[]]
                ranks_deg[graph] = [[]]
                scores_rand[graph] = [[]]
                ranks_rand[graph] = [[]]
                current = graph
                sample = 0
            else:
                sample += 1
                init_scores[graph].append([])
                init_ranks[graph].append([])
                scores_greedy[graph].append([])
                ranks_greedy[graph].append([])
                scores_bc[graph].append([])
                ranks_bc[graph].append([])
                scores_deg[graph].append([])
                ranks_deg[graph].append([])
                scores_rand[graph].append([])
                ranks_rand[graph].append([])
        else:
            fields = line.split(",")
            nodes[graph] = float(fields[1])
            edges[graph] = float(fields[2])
            init_scores[graph][sample].append(float(fields[3]))
            init_ranks[graph][sample].append(float(fields[4]))
            scores_greedy[graph][sample].append(float(fields[5]))
            ranks_greedy[graph][sample].append(float(fields[6]))
            scores_deg[graph][sample].append(float(fields[7]))
            ranks_deg[graph][sample].append(float(fields[8]))
            scores_bc[graph][sample].append(float(fields[9]))
            ranks_bc[graph][sample].append(float(fields[10]))
            scores_rand[graph][sample].append(float(fields[11]))
            ranks_rand[graph][sample].append(float(fields[12]))
    return nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand

In [ ]:
file = "../scripts/resultsCompHeuristicsDirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)

In [ ]:
# plot with betweenness of a single node for different values of k (one specific sample) -- Fig. 7 and 8

def plotBCImprovement(graph, sample, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand):
    figure(figsize=(6,3))
    x1 = array(range(0, 11))

    y1 = array([init_scores[graph][sample][0]]+scores_greedy[graph][sample])

    y2 = array([init_scores[graph][sample][0]]+scores_deg[graph][sample])
    y3 = array([init_scores[graph][sample][0]]+scores_bc[graph][sample])
    y4 = array([init_scores[graph][sample][0]]+scores_rand[graph][sample])

    p1 = plot(x1, y1, 'ro')
    p2 = plot(x1, y2, 'ro', color='#1B2ACC')
    p3 = plot(x1, y3, 'ro', color='#1A2A11')
    p4 = plot(x1, y4, 'ro', color='#AA21BB')
    legend((p1[0], p2[0], p3[0], p4[0]), ("Greedy","Top-k Degree", "Top-k Betweenness", "Random"), loc="best")

    xlim(-0.2, 10.2)
    xlabel('k')
    ylabel('Betweenness')
    show()

In [ ]:
# Figure 7 (left)
graph = "munmun_digg_reply"
sample = 0
plotBCImprovement(graph, sample, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)

In [ ]:
# Figure 7 (right)
graph = "munmun_digg_reply"
sample = 1
plotBCImprovement(graph, sample, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)

In [ ]:
# Figure 8 (left)
graph = "linux"
sample = 0
plotBCImprovement(graph, sample, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)

In [ ]:
# Figure 8 (right)
graph = "linux"
sample = 1
plotBCImprovement(graph, sample, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)

In [ ]:
# cumulative results: given a graph, we compute the average betweenness over all samples
def computeCumulativeResults(init_scores, scores_greedy, scores_deg, scores_bc, scores_rand):
    avg_scores_greedy = {}
    avg_scores_deg = {}
    avg_scores_bc = {}
    avg_scores_rand = {}
    avg_init_scores = {}

    for graph in scores_greedy:
        avg_init_scores[graph] = []
        avg_scores_greedy[graph] = []
        avg_scores_deg[graph] = []
        avg_scores_bc[graph] = []
        avg_scores_rand[graph] = []
        for k in range(10):
            sumk = sum([init_scores[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_init_scores[graph].append(sumk)
            sumk = sum([scores_greedy[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_scores_greedy[graph].append(sumk)
            sumk = sum([scores_deg[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_scores_deg[graph].append(sumk)
            sumk = sum([scores_bc[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_scores_bc[graph].append(sumk)
            sumk = sum([scores_rand[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_scores_rand[graph].append(sumk)
    return avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores

In [ ]:
def plotBCImprovementCumulative(graph, nodes, edges, avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores):
    figure(figsize=(6,3))
    x1 = array(range(0, 11))
    y1 = array([avg_init_scores[graph][0]]+avg_scores_greedy[graph])
    y2 = array([avg_init_scores[graph][0]]+avg_scores_deg[graph])
    y3 = array([avg_init_scores[graph][0]]+avg_scores_bc[graph])
    y4 = array([avg_init_scores[graph][0]]+avg_scores_rand[graph])
    y1 = [score/(nodes[graph]*nodes[graph]-edges[graph])*100 for score in y1]
    y2 = [score/(nodes[graph]*nodes[graph]-edges[graph])*100 for score in y2]
    y3 = [score/(nodes[graph]*nodes[graph]-edges[graph])*100 for score in y3]
    y4 = [score/(nodes[graph]*nodes[graph]-edges[graph])*100 for score in y4]

    p1 = plot(x1, y1, 'ro')
    p2 = plot(x1, y2, 'ro', color='#1B2ACC')
    p3 = plot(x1, y3, 'ro', color='#1A2A11')
    p4 = plot(x1, y4, 'ro', color='#AA21BB')
    legend((p1[0], p2[0], p3[0], p4[0]), ("Greedy","Top-k Degree", "Top-k Betweenness", "Random"), loc="best")

    xlim(-0.2, 10.2)
    xlabel('k')
    ylabel('Betweenness')
    show()

In [ ]:
# Figure 9 (left)

avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores = computeCumulativeResults(init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)
graph = 'munmun_digg_reply'
plotBCImprovementCumulative(graph, nodes, edges, avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores)

In [ ]:
# Figure 9 (right)

avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores = computeCumulativeResults(init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)
graph = 'linux'
plotBCImprovementCumulative(graph, nodes, edges, avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores)

In [ ]:
#average over all graphs -- Figure 11 (left) and Figure 12 (left)
def plotBCImprovementAllGraphs(nodes, edges, avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores):
    figure(figsize=(6,3))
    x1 = array(range(0, 11))
    labels = avg_init_scores
    y1 = [numpy.mean([avg_init_scores[graph][0]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels])]
    y2 = [numpy.mean([avg_init_scores[graph][0]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels])]
    y3 = [numpy.mean([avg_init_scores[graph][0]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels])]
    y4 = [numpy.mean([avg_init_scores[graph][0]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels])]
    for k in range(0,10):
        y1.append(numpy.mean([avg_scores_greedy[graph][k]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels]))
        y2.append(numpy.mean([avg_scores_deg[graph][k]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels]))
        y3.append(numpy.mean([avg_scores_bc[graph][k]/(nodes[graph]*nodes[graph]-edges[graph]-edges[graph])*100 for graph in labels]))
        y4.append(numpy.mean([avg_scores_rand[graph][k]/(nodes[graph]*nodes[graph]-edges[graph])*100 for graph in labels]))
    y1 = array(y1)
    y2 = array(y2)
    y3 = array(y3)
    y4 = array(y4)
    p1 = plot(x1, y1, 'ro')
    p2 = plot(x1, y2, 'ro', color='#1B2ACC')
    p3 = plot(x1, y3, 'ro', color='#1A2A11')
    p4 = plot(x1, y4, 'ro', color='#AA21BB')
    legend((p1[0], p2[0], p3[0], p4[0]), ("Greedy","Top-k Degree", "Top-k Betweenness", "Random"), loc="best")
    #ylim(1,1.3)
    xlabel('k')
    ylabel('Betweenness')
    xlim(-0.2, 10.2)
    show()

In [ ]:
#Figure 11 (left)
file = "../scripts/resultsCompHeuristicsDirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)
avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores = computeCumulativeResults(init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)
plotBCImprovementAllGraphs(nodes, edges, avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores)

In [ ]:
#Figure 12 (left)
# now we have to recompute everything for undirected graphs
file = "../scripts/resultsCompHeuristicsUndirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)
avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores = computeCumulativeResults(init_scores, scores_greedy, scores_deg, scores_bc, scores_rand)
plotBCImprovementAllGraphs(nodes, edges, avg_scores_greedy, avg_scores_deg, avg_scores_bc, avg_scores_rand, avg_init_scores)

In [ ]:
# cumulative results ranking: given a graph, we compute the average rank over all samples
def computeCumulativeResultsRank(init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand):
    avg_ranks_greedy = {}
    avg_ranks_deg = {}
    avg_ranks_bc = {}
    avg_ranks_rand = {}
    avg_init_ranks = {}

    for graph in ranks_greedy:
        avg_init_ranks[graph] = []
        avg_ranks_greedy[graph] = []
        avg_ranks_deg[graph] = []
        avg_ranks_bc[graph] = []
        avg_ranks_rand[graph] = []
        for k in range(10):
            sumk = sum([init_ranks[graph][sample][k] for sample in range(len(ranks_greedy[graph]))])
            sumk = sumk/float(len(ranks_greedy[graph]))
            avg_init_ranks[graph].append(sumk)
            sumk = sum([ranks_greedy[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_ranks_greedy[graph].append(sumk)
            sumk = sum([ranks_deg[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_ranks_deg[graph].append(sumk)
            sumk = sum([ranks_bc[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_ranks_bc[graph].append(sumk)
            sumk = sum([ranks_rand[graph][sample][k] for sample in range(len(scores_greedy[graph]))])
            sumk = sumk/float(len(scores_greedy[graph]))
            avg_ranks_rand[graph].append(sumk)
    return avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks

In [ ]:
# plot rank of a single node for different values of k (average over differnt samples)
def plotRankImprovementCumulative(graph, nodes, avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks):
    figure(figsize=(6,3))
    x1 = array(range(0, 11))
    y1 = array([avg_init_ranks[graph][0]]+avg_ranks_greedy[graph])
    y2 = array([avg_init_ranks[graph][0]]+avg_ranks_deg[graph])
    y3 = array([avg_init_ranks[graph][0]]+avg_ranks_bc[graph])
    y4 = array([avg_init_ranks[graph][0]]+avg_ranks_rand[graph])
    y1 = [score/nodes[graph]*100 for score in y1]
    y2 = [score/nodes[graph]*100 for score in y2]
    y3 = [score/nodes[graph]*100 for score in y3]
    y4 = [score/nodes[graph]*100 for score in y4]

    p1 = plot(x1, y1, 'ro')
    p2 = plot(x1, y2, 'ro', color='#1B2ACC')
    p3 = plot(x1, y3, 'ro', color='#1A2A11')
    p4 = plot(x1, y4, 'ro', color='#AA21BB')
    legend((p1[0], p2[0], p3[0], p4[0]), ("Greedy","Top-k Degree", "Top-k Betweenness", "Random"), loc="best")
    xlabel('k')
    xlim(-0.2, 10.2)
    ylabel('Rank')
    show()

In [ ]:
# Figure 10 (left)
file = "../scripts/resultsCompHeuristicsDirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)
avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks = computeCumulativeResultsRank(init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand)
graph = 'munmun_digg_reply'
plotRankImprovementCumulative(graph, nodes, avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks)

In [ ]:
# Figure 10 (right)
file = "../scripts/resultsCompHeuristicsDirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)
avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks = computeCumulativeResultsRank(init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand)
graph = 'linux'
plotRankImprovementCumulative(graph, nodes, avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks)


In [ ]:
#average ranking over all graphs -- Figure 11 (right) and Figure 12 (right)
def plotRankImprovementAllGraphs(nodes, edges, avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks):
    figure(figsize=(6,3))
    x1 = array(range(0, 11))
    labels = nodes
    y1 = [numpy.mean([avg_init_ranks[graph][0]/nodes[graph]*100 for graph in labels])]
    y2 = [numpy.mean([avg_init_ranks[graph][0]/nodes[graph]*100 for graph in labels])]
    y3 = [numpy.mean([avg_init_ranks[graph][0]/nodes[graph]*100 for graph in labels])]
    y4 = [numpy.mean([avg_init_ranks[graph][0]/nodes[graph]*100 for graph in labels])]
    for k in range(0,10):
        y1.append(numpy.mean([avg_ranks_greedy[graph][k]/nodes[graph]*100 for graph in labels]))
        y2.append(numpy.mean([avg_ranks_deg[graph][k]/nodes[graph]*100 for graph in labels]))
        y3.append(numpy.mean([avg_ranks_bc[graph][k]/nodes[graph]*100 for graph in labels]))
        y4.append(numpy.mean([avg_ranks_rand[graph][k]/nodes[graph]*100 for graph in labels]))

    y1 = array(y1)
    y2 = array(y2)
    y3 = array(y3)
    y4 = array(y4)
    p1 = plot(x1, y1, 'ro')
    p2 = plot(x1, y2, 'ro', color='#1B2ACC')
    p3 = plot(x1, y3, 'ro', color='#1A2A11')
    p4 = plot(x1, y4, 'ro', color='#AA21BB')
    legend((p1[0], p2[0], p3[0], p4[0]), ("Greedy","Top-k Degree", "Top-k Betweenness", "Random"), loc="best")
    xlabel('k')
    xlim(-0.2, 10.2)
    ylabel('Rank')

    show()

In [ ]:
# Figure 11 (right)
file = "../scripts/resultsCompHeuristicsDirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)
avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks = computeCumulativeResultsRank(init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand)
plotRankImprovementAllGraphs(nodes, edges, avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks)

In [ ]:
# Figure 12 (right)
file = "../scripts/resultsCompHeuristicsUndirected.txt"
nodes, edges, init_scores, scores_greedy, scores_deg, scores_bc, scores_rand, init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand = readDataHeuristics(file)
avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks = computeCumulativeResultsRank(init_ranks, ranks_greedy, ranks_deg, ranks_bc, ranks_rand)
plotRankImprovementAllGraphs(nodes, edges, avg_ranks_greedy,avg_ranks_deg,avg_ranks_bc,avg_ranks_rand,avg_init_ranks)